In [1]:
import pandas as pd
import numpy as np
import datetime as dt

from xbbg import blp

In [ ]:
# Time Period of Events
TODAY_DATE = pd.to_datetime('today')
END_OF_WEEK = (pd.datetime('today') + pd.DateOffset(days = (4 - TODAY_DATE.weekday())))

# Minimum Market Cap
MINIMUM_MKT_CAP = 1e11

In [ ]:
# Pull all tickers for the S&P 500
def get_sp500_tickers():
    sp500_constituents  = blp.bds('SPX INDEX', 
                                  'INDX_MEMBERS', 
                                  'DVD_Start_Dt' = dt.datetime.today().strftime('%Y%m%d'), 
                                  'DVD_End_Dt' = dt.datetime.today().strftime('%Y%m%d')
                                 )
    bbg_names = [ticker + ' Equity' for ticker in bbg_names]
    return bbg_names

# Pull all of the S&P 500 earnings events 
def get_sp500_earnings_this_week():
    sp500_tickers = get_sp500_tickers()
    
    earnings_data = blp.bdp(sp500_tickers, 
                            ['SECURITY_NAME', 
                             'TICKER', 
                             'EXPECTED_REPORT_DT', 
                             'EXPECTED_REPORT_TIME', 
                             'CUR_MKT_CAP']
                           )
    
    earnings_data['expected_report_dt'] = pd.to_datetime(earnings_data['expected_report_dt'])
    
    filter = earnings_data['expected_report_dt'] <= pd.Timestamp(END_OF_WEEK)
    filter &= earnings_data['expected_report_dt'] >= pd.Timestamp(TODAY_DATE)
    filter &= earnings_data['cur_mkt_cap'] >= MINIMUM_MKT_CAP
    filter &= earnings_data['cur_mkt_cap'] >= MINIMUM_MKT_CAP
    earnings_data = earnings_data[filter]
    
    earnings_data['cur_mkt_cap'] = (earnings_data['cur_mkt_cap']/(1e9)).astype(int).apply(lambda x:"{:,}".format(x))
    earnings_data['Day'] = pd.to_datetime(earnings_data['expected_report_dt']).dt.day_name()
    earnings_data.sort_values(['expected_report_dt', 'cur_mkt_cap'], ascending = [True, False], inplace = True)
    earnings_data.reset_index(drop = True, inplace = True)
    earnings_data.rename(columns = {'security_name': 'Company', 
                                    'ticker':'Ticker', 
                                    'expected_report_dt':'Earnings Date', 
                                    'expected_report_time':'Release Time (Eastern)', 
                                    'cur_mkt_cap':'Market Cap (B)'}, 
                         inplace = True)
    earnings_data = earnings_data[['Company', 
                                   'Ticker', 
                                   'Earnings Date', 
                                   'Day', 
                                   'Release Time (Eastern)', 
                                   'Market Cap (B)']]
    
    return earnings_data


def get_fed_hikes():
    fed_hike = blp.bdp(['FDTR Index'], ['ECO_RELEASE_DT', 'ECO_RELEASE_TIME'])
    fed_hike = fed_hike[(fed_hike['eco_release_dt'] <= END_OF_WEEK) 
                        & (fed_hike['eco_release_dt'] >= TODAY_DATE)]
    fed_hike['Company'] = 'FOMC Rate Decision'
    fed_hike['Ticker'] = 'FED'
    fed_hike['Day'] = pd.to_datetime(fed_hike['expected_report_dt']).dt.day_name()
    fed_hike['Market Cap (B)'] = np.inf
    
    fed_hike.rename(columns = {'eco_release_dt':'Earnings Date', 
                               'eco_release_time':'Release Time (Eastern)'}, 
                    inplace = True
                   )
    
    return fed_hike


def get_tsy_auctions():
    
    tsy_auctions = blp.bdp(['USB2YBC Index', 
                            'USB5YBC Index', 
                            'USN10YBC Index', 
                            'USBD30YBC Index'], 
                           ['ECO_RELEASE_DT', 
                            'ECO_RELEASE_TIME']
                          )
    tsy_auctions['Company'] = 'TSY Auction'
    tsy_auctions['Ticker'] = ['2Y', '5Y', '10Y', '30Y']
    tsy_auctions['Day'] = pd.to_datetime(tsy_auctions['eco_release_dt']).dt.day_name()
    tsy_auctions['Market Cap (B)'] = np.inf
    tsy_auctions = tsy_auctions[(tsy_auctions['eco_release_dt'] <= END_OF_WEEK) 
                                & (tsy_auctions['eco_release_dt'] >= TODAY_DATE)]
    tsy_auctions.rename(columns = {'eco_release_dt':'Earnings Date', 
                                   'eco_release_time':'Release Time (Eastern)'}, 
                        inplace = True
                       )
    
    if (len(tsy_auctions) == 0):
        return None
    
    tsy_amounts = blp.bdp(['WIT2 Govt', 
                           'WIT5 Govt', 
                           'WIT10 Govt', 
                           'WIT30 Govt'], 
                          ['BQ_FACE_AMT', 
                           'MTY_YEARS', 
                           'AMT_ISSUED']
                         )
    tsy_amounts.reset_index(inplace = True)
    tsy_amounts.columns = ['Ticker', 'Face', 'Years', 'Notional']
    tsy_amounts = tsy_amounts[['Ticker', 'Notional']]
    tsy_amounts['Ticker'] = ['10Y', '2Y', '30Y', '5Y']
    tsy_amounts['Notional'] = (tsy_amounts['Notional']/(1e9))
    tsy_amounts.columns = ['Ticker', 'Market Cap (B)']
    
    tsy_auctions = tsy_auctions.merge(tsy_amounts, how = 'left', on = 'Ticker', suffixes = ('_', ''))
    tsy_auctions = tsy_auctions.drop('Market Cap (B)_', axis = 1)
    tsy_auctions['Market Cap (B)'] = tsy_auctions['Market Cap (B)'].astype(int)
    
    return tsy_auctions

In [ ]:
earnings = get_sp500_earnings_this_week()
fed_hike = get_fed_hikes()
tsy_auction = get_tsy_auctions()
all_events = pd.concat([earnings, fed_hike, tsy_auction])
all_events.sort_values(['Earnings Date', 'Market Cap (B)'], ascending = [True, False])
all_events.reset_index(drop = True, inplace = True)

In [ ]:
all_events